<a href="https://colab.research.google.com/github/pkams/projetos_iniciais/blob/master/prevendo_notas_ENEM_com_diferentes_regressores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Importando bibliotecas básicas

import pandas as pd
import numpy as np

In [0]:
# O objetivo deste dataset é prever as notas de matemática do ENEM baseando-se nas outras features.

df = pd.read_csv('https://raw.githubusercontent.com/wesleykaoru/Codenation/master/Data_Science/datasets/train.csv')

In [162]:
df.head()

,Unnamed: 0,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,CO_MUNICIPIO_NASCIMENTO,NO_MUNICIPIO_NASCIMENTO,CO_UF_NASCIMENTO,SG_UF_NASCIMENTO,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,CO_ESCOLA,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,CO_UF_ESC,SG_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC,IN_BAIXA_VISAO,IN_CEGUEIRA,IN_SURDEZ,IN_DEFICIENCIA_AUDITIVA,IN_SURDO_CEGUEIRA,IN_DEFICIENCIA_FISICA,IN_DEFICIENCIA_MENTAL,IN_DEFICIT_ATENCAO,IN_DISLEXIA,IN_DISCALCULIA,IN_AUTISMO,...,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,Q026,Q027,Q028,Q029,Q030,Q031,Q032,Q033,Q034,Q035,Q036,Q037,Q038,Q039,Q040,Q041,Q042,Q043,Q044,Q045,Q046,Q047,Q048,Q049,Q050
0,1,ed50e8aaa58e7a806c337585efee9ca41f1eb1ad,2016,4314902,Porto Alegre,43,RS,24,M,0.0,1,1,4314902.0,Porto Alegre,43.0,RS,1,4,1,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,...,A,B,A,A,A,A,A,A,B,A,A,C,A,A,A,C,H,E,5.0,5.0,1.0,0.0,0.0,5,5,0,5,5,5,5,5.0,A,A,A,A,A,A,A,B,D
1,2,2c3acac4b33ec2b195d77e7c04a2d75727fad723,2016,2304707,Granja,23,CE,17,F,0.0,3,1,2304707.0,Granja,23.0,CE,2,0,2,1.0,0,23005157.0,2304707.0,Granja,23.0,CE,2.0,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0,...,A,B,A,A,A,A,A,A,B,A,A,A,A,A,A,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,5,1,1,1,1,1,NaN,A,A,C,A,B,A,A,C,A
2,3,f4545f8ccb9ff5c8aad7d32951b3f251a26e6568,2016,2304400,Fortaleza,23,CE,21,F,0.0,3,1,2304400.0,Fortaleza,23.0,CE,3,0,1,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,...,A,B,A,A,A,A,A,A,B,A,A,B,A,A,A,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,5,5,5,5,5,5,NaN,A,A,A,A,C,A,A,B,A
3,4,3d6ec248fef899c414e77f82d5c6d2bffbeaf7fe,2016,3304557,Rio de Janeiro,33,RJ,25,F,0.0,0,1,3304557.0,Rio de Janeiro,33.0,RJ,1,9,1,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,...,A,B,B,B,A,B,A,B,C,A,B,D,B,C,B,C,F,C,2.0,0.0,5.0,5.0,5.0,5,5,5,5,2,5,5,5.0,C,A,A,A,A,D,A,A,A
4,5,bf896ac8d3ecadd6dba1dfbf50110afcbf5d3268,2016,1302603,Manaus,13,AM,28,M,0.0,2,1,1505304.0,Oriximiná,15.0,PA,1,4,1,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,...,A,B,A,B,A,A,A,A,B,A,A,B,A,A,A,B,F,D,5.0,4.0,3.0,5.0,4.0,5,5,4,3,1,4,5,NaN,A,A,A,A,A,A,A,A,A


In [153]:
# Objetivo aqui é pegar as 10 váriaveis mais correlacionadas com váriavel de NU_NOTA_MT que é nosso alvo.

colunas = df.corr().nlargest(11, 'NU_NOTA_MT')['NU_NOTA_MT'].keys()
colunas

Index(['NU_NOTA_MT', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC',
       'NU_NOTA_REDACAO', 'TP_DEPENDENCIA_ADM_ESC', 'NU_NOTA_COMP3',
       'NU_NOTA_COMP5', 'NU_NOTA_COMP4', 'NU_NOTA_COMP2', 'NU_NOTA_COMP1'],
      dtype='object')

In [0]:
# Tiramos a váriavel alvo da lista e ficamos com as restantes

colunas = colunas[1:]

In [8]:
df[colunas].shape

(13730, 10)

In [0]:
# Por observação das váriaveis temos provas com valores NaN, o que pode indicar que a pessoa faltou a prova. 
# Supomos que alguém que faltou no dia (ou em ambos os dias) tirou nota baixa e/ou 0.
# Com essa suposição jogamos valores zerados para preencher os NaN's. 

# Salvamos o X e y

X = df[colunas].fillna(0)
y = df['NU_NOTA_MT']
y = y.fillna(0)

In [157]:
# Verificamos se tem algum NaN restante

X.isna().sum().sum(), y.isna().sum().sum()

(0, 0)

In [0]:
# Dividimos em treino e teste

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [0]:
# Importamos bibliotecas para testar diferentes hiperparametros
# Tedtaremos diferentes ordens de features com PolynomialFeatures

from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

In [0]:
# Aplicamos StandardScaler para normalizar

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [0]:
# Criamos os steps e fazemos Pipeline, também criamos uma lista de paramos a ser testada com GridSearch

steps = [('Poly',PolynomialFeatures()),('LinearRegression', LinearRegression())]
pipeline = Pipeline(steps)
params = {'Poly__degree':[1, 2, 3, 4]}
gs_cv = GridSearchCV(pipeline, param_grid = params, verbose=1, cv=5)

In [106]:
# Aplicamos GridSearch nos dados de treino

gs_cv.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    6.9s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('Poly',
                                        PolynomialFeatures(degree=2,
                                                           include_bias=True,
                                                           interaction_only=False,
                                                           order='C')),
                                       ('LinearRegression',
                                        LinearRegression(copy_X=True,
                                                         fit_intercept=True,
                                                         n_jobs=None,
                                                         normalize=False))],
                                verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'Poly__degree': [1, 2, 3, 4]}, pre_dispatch='2*n_jobs',
             refit=True, return_train_scor

In [107]:
# Verificamos o melhor parametro encontrado, como esperado uma grandeza maior que 1

gs_cv.best_params_

{'Poly__degree': 3}

In [108]:
# Qual melhor score obtido em treino

gs_cv.best_score_

0.9185205443440221

In [109]:
# Para verificar o estimador

gs_cv.best_estimator_

Pipeline(memory=None,
         steps=[('Poly',
                 PolynomialFeatures(degree=3, include_bias=True,
                                    interaction_only=False, order='C')),
                ('LinearRegression',
                 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
                                  normalize=False))],
         verbose=False)

In [0]:
# Podemos aplicar na mão os polynomials features

poly = PolynomialFeatures(degree=3)

In [0]:
# Aplicar sobre X_train e X_test essas features polinomiais

X_train_tran = poly.fit_transform(X_train)
X_test_tran = poly.fit_transform(X_test)

In [112]:
# Verificamos nosso resultado de Regressão Linear Multipla com SkLearn

linreg = LinearRegression()
linreg.fit(X_train_tran, y_train)

linreg.score(X_train_tran, y_train), linreg.score(X_test_tran, y_test)

(0.9260135498135377, 0.9184541728372874)

In [116]:
# Podemos testar também com RandomForestRegressor

from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor()

rfr.fit(X_train, y_train)

rfr.score(X_train, y_train)

0.9884225612567528

In [117]:
# Podemos perceber que houve overfitting devido a acurácia alta em treino porém mais baixa em teste

rfr.score(X_test, y_test)

0.9168892832073947

In [141]:
# Podemos testar também utilizando uma rede neural simples linear

import tensorflow as tf

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(100, input_shape=(10,)))
model.add(tf.keras.layers.Dense(100, 'relu'))
model.add(tf.keras.layers.Dense(1, 'linear'))

model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 100)               1100      
_________________________________________________________________
dense_16 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 101       
Total params: 11,301
Trainable params: 11,301
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [144]:
epochs_hist = model.fit(X_train, y_train, epochs=50)

Epoch 1/50
301/301 [==============================] - 1s 2ms/step - loss: 4397.2012
Epoch 2/50
301/301 [==============================] - 1s 2ms/step - loss: 4356.5000
Epoch 3/50
301/301 [==============================] - 1s 2ms/step - loss: 4336.4541
Epoch 4/50
301/301 [==============================] - 1s 2ms/step - loss: 4305.5474
Epoch 5/50
301/301 [==============================] - 1s 2ms/step - loss: 4282.1084
Epoch 6/50
301/301 [==============================] - 1s 2ms/step - loss: 4263.6421
Epoch 7/50
301/301 [==============================] - 1s 2ms/step - loss: 4212.8477
Epoch 8/50
301/301 [==============================] - 1s 2ms/step - loss: 4221.1978
Epoch 9/50
301/301 [==============================] - 1s 2ms/step - loss: 4194.1001
Epoch 10/50
301/301 [==============================] - 1s 2ms/step - loss: 4182.9517
Epoch 11/50
301/301 [==============================] - 1s 2ms/step - loss: 4175.8857
Epoch 12/50
301/301 [==============================] - 1s 2ms/step - loss:

In [0]:
ypred = model.predict(X_train)
ypred2 = model.predict(X_test)

In [0]:
from sklearn.metrics import r2_score

In [148]:
r2_score(y_train, ypred)

0.923032408636567

In [150]:
r2_score(y_test, ypred2)

0.9226709806350749

In [0]:
# Nosso resultado mais balanceado foi com a Rede Neural, mas o regressor linear também obteve bons resultados.
# Temos muitos pontos a melhorar principalmente utilizando métodos mais apropriados de exploração de dados 
# e também achar meios melhores de lidar com os NaN's.

